# A possible way of finding clusters using buffers

Clusters5 takes all of the data at face value and organizes it into an array. This code uses the buffers and goes through the buffers one at a time

Import Acqu and Timepix

In [1]:
import Acqu as aq
import Timepix
import numpy as np
import ROOT
from rootpy.plotting import Hist, histogram, Canvas, Hist2D

Welcome to JupyROOT 6.16/00


Import data and open file

In [2]:
inFile = '/w/work0/mainz/2019_05_Timepix3-Acqu/Timepix_33.dat'
aq.openFile(inFile)

Mk2 Data


Take the data from each detector, A and B.

In [3]:
hitsA = 0
hitsB = 0
TimepixAData = []
TimepixBData = []

def readTimepixData():
    if(aq.epicsEvent==1):
        nHitsA       = aq.getEpicsPV('PPOL:TIMEPIXA:NHITS')  #blen for A
        encodedA     = aq.getEpicsPV('PPOL:TIMEPIXA:ENCODED')
        nHitsB       = aq.getEpicsPV('PPOL:TIMEPIXB:NHITS')  #readTimepixBufferFromFile
        encodedB     = aq.getEpicsPV('PPOL:TIMEPIXB:ENCODED')
        TimepixAData.append(Timepix.Decode(nHitsA,encodedA))
        TimepixBData.append(Timepix.Decode(nHitsB,encodedB))
        global hitsA
        hitsA+=nHitsA
        global hitsB
        hitsB+=nHitsB
        
       
aq.runFunction(readTimepixData,0,50000)
print(hitsA,hitsB)

(65911.0, 46085.0)


Code made to loop through and array and take certain value. Here it is ToA for both A and B data. Need to organize before just taking out data.

Here, the data is organized by ToA, ToT, x-position, y-position. Made a new array that has each data point with those three values.

(have to run through twice bc first seperates all the data, second foor loop takes out each specefic piece)

In [4]:
xAData = []
xA = []

yAData = []
yA = []

ToAAData = []
ToAA = []

ToTAData = []
ToTA = []


dataA = []

for n in range(len(TimepixAData)):
    ToAAData.append(TimepixAData[n]['ToA'])  #Of that position so you get the apropriate corresponding 
    ToTAData.append(TimepixAData[n]['ToT'])  #ToT, x and y values
    xAData.append(TimepixAData[n]['x'])
    yAData.append(TimepixAData[n]['y'])  #now have whole list of all ToA, ToT, etc values individualy
    
for i in range(len(ToAAData)):
    for m in range(len(ToAAData[i])):
        ToAA.append(ToAAData[i][m])
        ToTA.append(ToTAData[i][m])
        xA.append(xAData[i][m])
        yA.append(yAData[i][m])
           
for h in range(len(ToAA)):
    dataA.append((ToAA[h],ToTA[h],xA[h],yA[h]))

dataA.sort()
#print("A Data: ")
#print(AData)        
#print("END")

###############################################
xBData = []
xB = []

yBData = []
yB = []

ToABData = []
ToAB = []

ToTBData = []
ToTB = []

dataB = []

for n in range(len(TimepixBData)):
    ToABData.append(TimepixBData[n]['ToA'])
    ToTBData.append(TimepixBData[n]['ToT'])
    xBData.append(TimepixBData[n]['x'])
    yBData.append(TimepixBData[n]['y'])
    
for i in range(len(ToABData)):
    for m in range(len(ToABData[i])):
        ToAB.append(ToABData[i][m])
        ToTB.append(ToTBData[i][m])
        xB.append(xBData[i][m])
        yB.append(yBData[i][m])

for h in range(len(ToAB)):
    dataB.append((ToAB[h],ToTB[h],xB[h],yB[h]))

dataB.sort()
#print("B Data: ")
#print(BData) 
#print("END")

Code for processing and graphing clusters.

Note that [0] is ToA, [1] is ToT, [2] is x and [3] is y.

In [5]:
def process_cluster(letter):
    start_index = cl[0]              #cl[0] buffer number at that position in array
    min_time = data[start_index][0]   #time of arrival of the first hit .ToA or [0]
    dtime = 0.0                          # time difference
    meanx = 0.0
    meany = 0.0
    
    #finding min_time in each cluster and the manx and meany
    n = start_index
    while(n<start_index+cn):            #loop ovber each hit in a cluster to find the lowest time
        if(data[n][0] < min_time):      #if this hit's time is lower than current min value
            min_time = data[n][0]       #set this lower value to min time
        meanx+=data[n][2]               # add up all the x positions
        meany+=data[n][3]               # add up all the y positions
        n+=1                         # move to the next hit in the cluster

    meanx/=cn        #divide sum of the positions by number of hits to get the mean 
    meany/=cn                        
    
    #plots 
    if(letter=='A'):
        tdposA.Fill(meanx,meany)    #mean x vs mean y plot
        multA.Fill(cn)             #Frequency of hits per cluster

        #plots
        n = start_index
        while(n<start_index+cn):               # for each hit in the cluster
            dtime = data[n][0] - min_time     # find time difference between a hit in the cluster and the min time of the cluster
            dthistA.Fill(dtime)                # put the time difference in the histogram
            dtime_ToTA.Fill(dtime,dataA[n][1])  #2D hist of time diff vs ToT for A data 
            n+=1  
    #plots   
    elif(letter=='B'):
        tdposB.Fill(meanx,meany)         # plot mean y vs. mean x
        multB.Fill(cn)                   # plot the counts in the cluster

        #plots
        n = start_index
        while(n<start_index+cn):               #loop through for each hit in the cluster
            dtime = data[n][0] - min_time     #time diff from earliest in cluster to each hit
            dthistB.Fill(dtime)                # put the time difference in the histogram
            dtime_ToTB.Fill(dtime,dataB[n][1]) #2D hist of time diff vs ToT for B data 
            n+=1                               #go on to the next hit in that same cluster


Code for identifying clusters with each detector's data. If hit is within

In [6]:
def cluster_finder(detector):

    global cl,cn,blen,b,nc,data,letter
    cl = [None]*100     #hold up to 100 hits in a cluster
    cn = 0            #counts number of hits in current cluster
    blen = 0            #length of the current timepix3 buffer
    b = 0               # the running (current?) buffer position
    nc = 0                  # count the number of clusters found
    
    if (detector=='A'):
        blen = hitsA              #read each timepix buffer into tp3 (number of entries)
        data = dataA
        letter = 'A'
    elif (detector=='B'):
        blen = hitsB
        data = dataB
        letter = 'B'

    while(b<blen):          #for every hit in the buffer (for every buffer position in buffer)
        cn = 0            # zero the cluster counter for the current buffer position
        cl[cn] = b   # assume the current position has the first hit in the cluster and save it
        cn+=1             # increment the counter, since we've counted a hit

        # this next bit was a for loop in Ken's code
        c=b+1             # start at position c that is one above the current buffer position
        while((c<b+99) and (c<blen)):     # check all positions up to 99 away, as long as we're still in the buffer
            if(abs(data[c][0]- data[b][0] < 100)):   # if an event at c is within 100ns of that at the current buffer position
                cl[cn] = c                             # save this position to the cluster
                cn+=1                                  # increment the counter, since we've counted a hit
            else:
                process_cluster(letter)               # analyze the cluster we found
                b+=cn                           # move on beyond this cluster
                cn = 0                          # reset the counter for hits in the cluster
                cl = [None]*100                 # reset the array of cluster hits
                nc+=1                           # increment the number of clusters found

                if(c<b+99 and c<blen):          # if we're still in the loop
                    cl[cn] = b                  # assume current position is the first in the new cluster
                    cn+=1                       # increment the counter, since we've counted a hit

            c+=1 # move to the next position and go through this loop again
            if(((c>=blen) and cn!=0)or((c>=b+99) and cn!=0)): # if you're exiting the loop but have a cluster to process still
                process_cluster(letter)
                b+=cn                  
                cn = 0                     
                cl = [None]*100         
                nc+=1
        b+=1

This plots all four graphs. Calls on cluster_finder() which then calls on the process_cluster().

A data graphs are all on one canvas and B data graphs are on two. 

In [7]:
ROOT.enableJSVis()
from ROOT import TF1

# plots for detector A

tdposA = Hist2D(256,0,256,256,0,256)
tdposA.SetTitle("tdposA; Mean x position of cluster; Mean y position of cluster")

multA = Hist(50,0,50)
multA.SetTitle("multA; Number of hits in a cluster; Number of clusters with that amount of hits")

dthistA = Hist(100,0,100)
dthistA.SetTitle("Time difference;Time difference between a hit and the min time of the cluster; Number of clusters")

dtime_ToTA = Hist2D(100,0,100,250,0,250)
dtime_ToTA.SetTitle("Time diff vs ToT; Time difference between minimum time in a cluster and a hit; ToT")


cA1 = ROOT.TCanvas("myA1","Cluster plots A1", 1000,1000)
cA1.Divide(1,2)
cA2 = ROOT.TCanvas("myA2","Cluster plots A2", 1000,1000)
cA2.Divide(1,2)

cluster_finder('A')


cA1.cd(1)
tdposA.Draw("colz")
cA1.cd(2)
multA.Fit("gaus")
multA.Draw()
cA1.Draw()
cA2.cd(1)
dthistA.Draw()

cA2.cd(2)
func = TF1('func',"gaus", 0, 100)
fit = dtime_ToTA.Fit('func', 'S')

dtime_ToTA.Draw()

#dtime_ToTA.Fit("gaus")
dtime_ToTA.Draw("colz")
cA2.Draw()

##############

 FCN=6297.79 FROM MIGRAD    STATUS=CALL LIMIT   1346 CALLS        1347 TOTAL
                     EDM=0.191917    STRATEGY= 1  ERROR MATRIX UNCERTAINTY  46.2 per cent
  EXT PARAMETER                APPROXIMATE        STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.01075e+07   1.83929e+06   4.03953e+04   1.20124e-06
   2  Mean        -4.32413e+01   9.73570e-01   4.07132e-02   5.15640e+00
   3  Sigma        1.12756e+01   1.44060e-01  -7.96539e-04   3.24691e+02

****************************************
Minimizer is Minuit / Migrad
Chi2                      =  1.16207e+08
NDf                       =         5389
Edm                       =   0.00701686
NCalls                    =          251
Constant                  =      59.1608   +/-   1.42437     
Mean                      =     -35.0324   +/-   49.181      
Sigma                     =      307.406   +/-   0.424886     	 (limited)


In [18]:
can = ROOT.TCanvas("can","okay", 1000,1000)
gr1 = ROOT.TGraph()


means = [0]*6
ents = [0]*6

x = []
y=[]

for b in range(1,7):
    for i in range(256):
        ents[b-1] += dtime_ToTA.GetBinContent(b,i)
        means[b-1] += (dtime_ToTA.GetBinContent(b,i))*(i-1)
    means[b-1]/=ents[b-1]
    x.append(b)
    y.append(means[b-1])
    gr1.SetPoint(b-1,x[b-1],y[b-1])


gr1.GetXaxis().SetRange(0,100)
gr1.GetYaxis().SetRange(0,256)
gr1.Draw("*")
gr1.Fit("gaus")

f6 = ROOT.TF1("f","222804*exp(-0.5*[(x+20.6389 )/5.42626]*[(x+20.6389 )/5.42626])",0,25)
#f6.Draw()
#  G(x) = c * exp ( -0.5 * [(x-m)/s]^2 ) 


can.Draw()

print(means)

[79.27912537451026, 32.86995635521511, 19.12805216426193, 10.108527131782946, 11.20855614973262, 45.711711711711715]

****************************************
         Invalid FitResult  (status = 4 )
****************************************
Minimizer is Minuit / Migrad
Chi2                      =      1881.13
NDf                       =            3
Edm                       =     0.503345
NCalls                    =         1353
Constant                  =  6.23629e+06   +/-   2.17727e+07 
Mean                      =     -49.1535   +/-   52.912      
Sigma                     =      10.5082   +/-   10.175       	 (limited)


In [14]:

#dtime_ToTA

#Fit slices projected along Y fron bins in X [0,7] with more than 20 bins in Y filled
#dtime_ToTA.FitSlicesY(0,0,7,20)
outfits = ROOT.TObjArray()
cA3 = ROOT.TCanvas("myA3","Least squares of gaussian 2D histogram A3", 800,800)
dtime_ToTA.FitSlicesY(0,0,4,4,"R",outfits)
dtime_ToTA.SetTitle("Time diff vs mean ToT; Time difference; mean ToT")
print dtime_ToTA.GetName()
dtime_ToTA.Fit("gaus")
outfits[1].Draw()

cA3.Draw()

Hist2D_bdkQAc77HmYbFqf6R6JyH7
 FCN=2485.78 FROM MIGRAD    STATUS=CONVERGED      75 CALLS          76 TOTAL
                     EDM=3.28501e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     3.46690e+02   2.30210e+00   4.70418e-02  -6.03451e-05
   2  Mean         7.95296e+01   2.42665e-01   5.40900e-03   2.98665e-04
   3  Sigma        3.77856e+01   1.56350e-01   2.62709e-05   1.23382e-01
 FCN=1112.58 FROM MIGRAD    STATUS=CONVERGED      73 CALLS          74 TOTAL
                     EDM=1.72301e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.96638e+02   2.68185e+00   3.18596e-02  -2.58486e-05
   2  Mean         2.73082e+01   3.20598e-01   4.50795e-03  -1.56266e-04
   

In [10]:
#Create a canvas and divide it
c1=ROOT.TCanvas("myA3","Least squares of gaussian 2D histogram A3",700,500)

c1.SetFillColor(42)
c1.Divide(2,1)
c1.cd(1)
leftPad = ROOT.TPad("leftPad","",0,0.94,1,1)
leftPad.Divide(1,2)

dtime_ToTA = Hist2D(100,0,100,250,0,250)
dtime_ToTA.SetTitle("Time diff vs ToT; Time difference between minimum time in a cluster and a hit; ToT")

#draw original 2D histogram
leftPad.cd(1)
gPad.SetTopMargin(0.12)
gpad.SetFillColor(33)
dtime_ToTA.Draw()
dtime_ToTA.GetXaxis().SetLabelSize(0.06)
dtime_ToTA.GetYaxis().SetLabelSize(0.06)
dtime_ToTA.SetMarkerColor(kYellow)

#Fit slices projected along Y fron bins in X [7,32]
dtime_ToTA.FitSlicesY(0,0,0,20)

#Show fitted "mean" for each slice
leftPad.cd(2)
gPad.SetFillColor(33)
dtime_ToTA_0.Draw()
c1.cd(2)
rightPad = ROOT.TPad.GPad("rightPad","",0,0.94,1,1)
rightPad.Divide(1,2)
rightPad.cd(1)
gPad.SetTopMargin(0.12)
gPad.SetLeftMargin(0.15)
gPad.SetFillColor(33)
dtime_ToTA_1.Draw()


#Show fitted "sigma" for each slice
rightPad.cd(2)
gPad.SetTopMargin(0.12)
gPad.SetLeftMargin(0.15)
gPad.SetFillColor(33)
dtime_ToTA_2.SetMinimum(0.8)
dtime_ToTA_2.Draw(

SyntaxError: invalid syntax (<ipython-input-10-4faece5eebde>, line 45)

In [ ]:
void FitSlicesY(TF1 *f1, Int_t binmin, Int_t binmax, Int_t cut, Option_t *option) 


In [ ]:
#plot for B data
##############

tdposB = Hist2D(256,0,256,256,0,256)
tdposB.SetTitle("tdposB; Mean x position of cluster; Mean y position of cluster")


multB = Hist(50,0,50)
multB.SetTitle("multB; Number of hits in a cluster; Number of clusters with taht amount of hits")


dthistB = Hist(100,0,100)
dthistB.SetTitle("Time difference;Time difference between a hit and the min time of the cluster; Number of clusters")


dtime_ToTB = Hist2D(100,0,100,250,0,250)
dtime_ToTB.SetTitle("Time diff vs ToT; Time difference between minimum time in a cluster and a hit; ToT")


cB1 = ROOT.TCanvas("myB1","Cluster plots B1", 1000,1000)
cB1.Divide(1,2)
cB2 = ROOT.TCanvas("myB2","Cluster plots B2", 1000,1000)
cB2.Divide(1,2)

cluster_finder('B')

cB1.cd(1)
tdposB.Draw("colz")
cB1.cd(2)
multB.Fit("gaus")
multB.Draw()
####################
cB2.cd(1)
dthistB.Fit("gaus")
dthistB.Draw()
cB2.cd(2)
#dtime_ToTB.Fit("gaus")
dtime_ToTB.Draw("colz")
cB1.Draw()
cB2.Draw()

# Graphing equations in jupyter notebook.